In [1]:
from collections import Counter
from functools   import cache
from itertools   import tee
from helpers     import data

In [2]:
adapter_joltages = data(10, parser=int)

In [3]:
adapter_joltages[:5]

[118, 14, 98, 154, 71]

**Part 1:** Multiply number of 1-jolt differences by number of 3-jolt differences. Any adapter can have input of 1,2,or 3 jolts lower than its output joltage. Our device has ouput joltage 3 jolts higher than maximum of adapter. The charging outlet has joltage 0. 

I assume there's only one way to order the adapters. 

This seems very simple: since an adapter can only take as input another adapter with input at least 1 and at most 3 joltages below it, we can just sort the list and use that order. 

In [4]:
device_joltage = max(adapter_joltages) + 3
input_joltage = 0

In [5]:
order = [input_joltage, *sorted(adapter_joltages), device_joltage]
order[:10]

[0, 1, 2, 5, 8, 9, 10, 11, 14, 15]

In [6]:
def pairwise(iterable):
    """s -> (s0, s1), (s1, s2), ..."""
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [7]:
diffs = [output_joltage - input_joltage for input_joltage, output_joltage in pairwise(order)]
if not all(1 <= d <= 3 for d in diffs):
    raise ValueError("Order is impossible!")

In [8]:
totals = Counter(diffs)
totals[1] * totals[3]

2070

I'm really happy with this solution! It didn't take me too long, and I think using `pairwise` makes it much easier. Of course, I didn't really do much since `sorted` just solves the problem. 

**Part 2:** Find distinct ways to arrange the adapters to charge your device. Now, we don't have to include them all.


This is still pretty simple, I think. It's got to still be in sorted order, so we can sort, then try removing one adapter, check if diff works, then try removing another one, and repeat until it no longer works. Then start again by removing another adapter. 

Actually, what we should do is try removing just one adapter. We'll have a list of all the adapters that, when removed individually, still allowed for a valid ordering. Now when we do multiple removals at a time, we can only remove those ones. 

Nah, this is just a simple "use the first one or don't". 

In [9]:
@cache
def arrangements(jolts, prev=0):
    """jolts must be a tuple so it can be cached."""
    first, rest = jolts[0], jolts[1:]
    if first - prev > 3:
        return 0 
    elif not rest:
        return 1 
    return arrangements(rest, first) + arrangements(rest, prev)

In [10]:
arrangements(tuple( sorted(adapter_joltages) + [device_joltage] ))

24179327893504

In [11]:
arrangements.cache_info()

CacheInfo(hits=72, misses=269, maxsize=None, currsize=269)